In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import sys
sys.path.append('..')

import pandas as pd

from cogwheel import posterior
from cogwheel import utils
from cogwheel import sampling
from cogwheel import data
from cogwheel import coherent_score

from pathlib import Path

In [ ]:
parentdir = Path('/data/jroulet/GW/cogwheel/')  # SET ME

In [ ]:
metadata = pd.read_csv(data.DATADIR/'events_metadata.csv', index_col=0)  # Chirp mass guesses

In [ ]:
eventnames = metadata.index.intersection(
    path.parent.name for path in (parentdir/'IASPrior').glob('*/Posterior.json'))

In [ ]:
for eventname in eventnames:
    old_post = utils.read_json(
        f'/data/jroulet/GW/cogwheel/IASPrior/{eventname}/Posterior.json')

    prior = coherent_score.intrinsic_prior.IntrinsicIASPrior(
        **old_post.prior.get_init_dict())
    
    like = coherent_score.CoherentScoreLikelihood(
        **old_post.likelihood.get_init_dict())

    post = posterior.Posterior(prior, like)
    pym = sampling.PyMultiNest(post)
    pym.run_kwargs['n_live_points'] = 1024
    rundir = pym.get_rundir(parentdir)
    pym.submit_slurm(rundir)